### 用tf.data加载图片数据
本教程提供一个如何使用 tf.data 加载图片的简单例子。  
本例中使用的数据集分布在图片文件夹中，一个文件夹含有一类图片。   
处理流程如下：  
+ 将数据集下载到本地，不同类别存储在不同文件夹中
+ 创建一个列表all_image_labels，包含每个文件的标签索引
+ 写函数用tf.io加载图片，用tf.image预处理图片
+ 构造一个tf.data.Dataset
+ 传递数据集至模型

In [1]:
# 添加了"/gpu:1"，在卷积运算时不会报错！！！！！
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

import numpy as np
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

#### 1. 获取图片数据集-->得到图片路径列表-->统计图片个数-->制作图片标签
数据集包含3670张5类花的图片，分别是：daisy,dandelion,roses,sunflower,tulips

In [2]:
# 下载数据集，返回的是存储在本地的、解压后的数据路径
data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', 
                                         untar=True)
print("各类flower所在父目录为：",data_root_orig)


import pathlib
import random
# 用pathlib这个包，获取所有图片的绝对路径
data_root = pathlib.Path(data_root_orig)    
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
print("First 10 labels indices: ", all_image_paths[:10])


# 统计图片的数目
image_count = len(all_image_paths)
print("数据集一共有{:04d}张图片".format(image_count))


# 列出可用的标签-->为每个标签分配索引-->创建一个列表包含每个文件的标签索引
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
print("First 10 labels indices: ", all_image_labels[:10])

各类flower所在父目录为： /root/.keras/datasets/flower_photos
First 10 labels indices:  ['/root/.keras/datasets/flower_photos/dandelion/3458770076_17ed3a1225.jpg', '/root/.keras/datasets/flower_photos/roses/16155980245_6ab8d7b888.jpg', '/root/.keras/datasets/flower_photos/dandelion/7015947703_11b30c20c9_n.jpg', '/root/.keras/datasets/flower_photos/sunflowers/24459548_27a783feda.jpg', '/root/.keras/datasets/flower_photos/sunflowers/2816256710_a2d3616fae.jpg', '/root/.keras/datasets/flower_photos/dandelion/4254850910_0610224342_n.jpg', '/root/.keras/datasets/flower_photos/sunflowers/184682652_c927a49226_m.jpg', '/root/.keras/datasets/flower_photos/dandelion/6994938270_bf51d0fe63.jpg', '/root/.keras/datasets/flower_photos/tulips/16702188449_3dacce90b2_m.jpg', '/root/.keras/datasets/flower_photos/dandelion/8956863946_f96be02aae_n.jpg']
数据集一共有3670张图片
First 10 labels indices:  [1, 2, 1, 3, 3, 1, 3, 1, 4, 1]


#### 2. 图片的加载（tf.io）-->预处理（tf.image）-->构建tf.data.Dataset
构建 tf.data.Dataset 最简单的方法就是使用 from_tensor_slices 方法。  
对dataset操作的注意事项：  
+ 在 .repeat 之后 .shuffle，会在 epoch 之间打乱数据（当有些数据出现两次的时候，其他数据还没有出现过）
+ 在 .batch 之后 .shuffle，会打乱 batch 的顺序，但是不会在 batch 之间打乱数据。
+ 在完全打乱中使用和数据集大小一样的 buffer_size（缓冲区大小）。较大的缓冲区大小提供更好的随机化，但使用更多的内存，直到超过数据集大小。
+ 在从随机缓冲区中拉取任何元素前，要先填满它。所以当你的 Dataset（数据集）启动的时候一个大的 buffer_size（缓冲区大小）可能会引起延迟。
+ 在随机缓冲区完全为空之前，被打乱的数据集不会报告数据集的结尾。Dataset（数据集）由 .repeat 重新启动，导致需要再次等待随机缓冲区被填满。

In [3]:
# 将单张图片的读取、解码、resize、归一化等操作，包装在一个简单的函数里备用
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    # normalize to [0,1] range
    image /= 255.0                                  
    return image


# 再包装一下，方便tf.data.Dataset对象调用map函数
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label


# 构造训练集和测试集的数据集
def dataset_construction(all_image_paths, all_image_labels, training_rate=0.9, BATCH_SIZE=64):
    training_nums = int(image_count*training_rate)
    # 将paths和labels切割为train和test部分
    train_image_paths, test_image_paths = np.split(all_image_paths, [training_nums,])
    train_image_labels, test_image_labels = np.split(all_image_labels, [training_nums,])
    # 将图片和标签数据集放到一块，用上面的函数处理每张图片路径
    ds_train = tf.data.Dataset.from_tensor_slices((train_image_paths, train_image_labels)).map(load_and_preprocess_from_path_label)
    ds_test = tf.data.Dataset.from_tensor_slices((test_image_paths, test_image_labels)).map(load_and_preprocess_from_path_label)
    # 为数据集在训练时设置一些参数
    # 设置一个和数据集大小一致的 shuffle buffer size（随机缓冲区大小）以保证数据被充分打乱
    # 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
    ds_train = ds_train.shuffle(buffer_size=image_count).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    ds_test = ds_test.shuffle(buffer_size=image_count).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    return ds_train, ds_test


ds_train, ds_test = dataset_construction(all_image_paths, all_image_labels)

#### 3. 模型的迁移，重新构造与编译，训练与保存，评估与预测

In [4]:
# 从 tf.keras.applications 取得 MobileNet v2 副本用于迁移学习
# 这是一个keras下.h5格式的模型，包括模型结构和模型参数
# 设置 MobileNet 的权重为不可训练：  
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable = False
mobile_net.summary()

Model: "mobilenetv2_1.00_192"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 193, 193, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 96, 96, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 96, 96, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
# 看一下该迁移模型的特点
help(tf.keras.applications.mobilenet_v2.preprocess_input)


# 该模型期望它的输出被标准化至 [-1,1] 范围内
def change_range(image,label):
  return 2*image-1, label
ds_train = ds_train.map(change_range)
ds_test = ds_test.map(change_range)


# MobileNet为每张图片的特征返回一个 6x6 的空间网格
# 数据集可能需要几秒来启动，因为要填满其随机缓冲区。
image_batch, label_batch = next(iter(ds_train))
print(image_batch.shape)
print(label_batch.shape)

# 检查一下最后一层输出的尺寸大小
feature_map_batch = mobile_net(image_batch)
print(feature_map_batch.shape)

Help on function wrapper in module tensorflow.python.keras.applications:

wrapper(*args, **kwargs)

(64, 192, 192, 3)
(64,)
(64, 6, 6, 1280)


In [6]:
# 构建完整模型：在MobileNet V2模型后面再加2层
model = tf.keras.Sequential([mobile_net,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(len(label_names), activation = 'softmax')])

# 看下完整模型的输出是个啥
logit_batch = model(image_batch).numpy()
print("min logit:", logit_batch.min())
print("max logit:", logit_batch.max())
print("Shape:", logit_batch.shape)

# 编译模型以描述训练过程
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

# 此处有两个可训练的变量 —— Dense 层中的 weights（权重） 和 bias（偏差）
len(model.trainable_variables)

# 看下模型长啥样
model.summary()

min logit: 0.0051106783
max logit: 0.9477112
Shape: (64, 5)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Model) (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [7]:
# 看完觉得没问题就可以开始训练
model.fit(ds_train, epochs=10, steps_per_epoch=50)

# 我们可以手动保存下最终模型
model.save('mobile_net_v2_1.0_192_flowers.h5')


Train for 50 steps
Epoch 1/10
50/50 [==============================] - 8s 168ms/step - loss: 0.9379 - accuracy: 0.6409
Epoch 2/10
 2/50 [>.............................] - ETA: 1:31 - loss: 0.6073 - accuracy: 0.7500

KeyboardInterrupt: 

In [ ]:
for step, (x, y) in enumerate(ds_test):
    predictions = model(x)
    predictions = tf.cast(tf.argmax(predictions, axis=1), tf.int64)
    print(predictions.shape)
    print(y.shape)
    print(predictions)
    print(y)
    #loss = tf.keras.metrics.Mean(y, predictions)
    accuracy = tf.keras.metrics.Accuracy(y, predictions)
    print("step:"+str(step), "loss:"+str(loss), "accuracy:"+str(accuracy))